In [1]:
#API
from flask import Flask, jsonify
from flask import request
from flasgger import Swagger, LazyString, LazyJSONEncoder
from flasgger import swag_from

#Cleansing
import re 
import pandas as pd
import numpy as np
import demoji
import pickle
import string

#SKLearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

#SQLite
import sqlite3

#defination of data test, train, and validation
df = pd.read_csv("train_preprocess.txt", delimiter = "\t", header=None)

df.columns =['text', 'label']
sentiment = ['negative', 'neutral', 'positive']

#ChangeAlayPath
df_kbbi = pd.read_csv('KamusAlayIvan.csv', header=None, encoding='ISO-8859-1', names=['TIDAKBAKU', 'BAKU'])

In [2]:
#DefinitionCleansingFunction
def removechars(text):
    text = re.sub(r'[^\w]', ' ', text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))','',text)
    text = re.sub('\d', '', text)
    text = re.sub(r'\b\w\b', '', text)
##Stopwords in RemoveChars
    text = re.sub('di','',text) 
    text = re.sub('yang','',text) 
    text = re.sub('dan','',text) 
    text = re.sub('nya','',text) 
    text = re.sub('saya','',text)
    text = re.sub('ini','',text)
    text = re.sub('itu','',text)
    text = re.sub('aku','',text)
    text = re.sub('kamu','',text)
    text = re.sub('th','',text)
    text = re.sub('sekali','',text)
    return text

def changealay(text):
    alay = dict(zip(df_kbbi['TIDAKBAKU'], df_kbbi['BAKU']))
    text = ' '.join([alay[word] if word in alay else word for word in text.split(' ')])
    return text

In [3]:
#CallingCleansingFunction

def cleaning(text):
    text = removechars(text)
    text = changealay(text)
    text = text.lower(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [4]:
df

,text,label
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
...,...,...
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive
10997,hormati partai-partai yang telah berkoalisi,neutral
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative


In [ ]:
    # save ke sql
conn = sqlite3.connect('datacsv_tosql1.db')
df.to_sql('Datanew', conn, if_exists='replace', index=False)

In [5]:
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(df['text'])

x = tfidf_vect.transform(df['text'])
print('Feature Extraction TFIDF Selesai')

Feature Extraction TFIDF Selesai


In [6]:
pickle.dump(tfidf_vect, open('tfidf_vect.pkl', 'wb'))
classes = df['label']
x_train, x_test, y_train, y_test = train_test_split(x, classes, test_size = 0.2, stratify=classes)
model = MLPClassifier()
model.fit(x_train, y_train)

print ('Training selesai')

Training selesai


In [7]:
classes

0        positive
1         neutral
2        positive
3        positive
4        negative
           ...   
10995    positive
10996    positive
10997     neutral
10998    negative
10999    positive
Name: label, Length: 11000, dtype: object

In [8]:
pickle.dump(model, open('model.p', 'wb'))
test = model.predict(x_test)

print ("Testing selesai")

print(classification_report(y_test, test))

Testing selesai
              precision    recall  f1-score   support

    negative       0.78      0.82      0.80       687
     neutral       0.80      0.68      0.73       230
    positive       0.89      0.89      0.89      1283

    accuracy                           0.84      2200
   macro avg       0.82      0.79      0.81      2200
weighted avg       0.85      0.84      0.84      2200



In [9]:
original_text =  '''
anak baik'''

# Feature Extraction
count_vect = TfidfVectorizer()
text = tfidf_vect.transform([(original_text)])

# Kita prediksi sentimennya
result = model.predict(text)[0]
print("Sentiment:")
print()
print(result)

Sentiment:

positive


In [10]:
original_text =  '''
anak setan
'''

# Feature Extraction
count_vect = TfidfVectorizer()
text = tfidf_vect.transform([(original_text)])

# Kita prediksi sentimennya
result = model.predict(text)[0]
print("Sentiment:")
print()
print(result)

Sentiment:

negative


In [11]:
original_text =  '''
anak setan
'''

# Feature Extraction
count_vect = TfidfVectorizer()
text = tfidf_vect.transform([(original_text)])

# Kita prediksi sentimennya
result = model.predict(text)[0]
print("Sentiment:")
print()
print(result)

Sentiment:

negative


In [12]:
original_text =  '''
mobil rusak
'''

# Feature Extraction
count_vect = TfidfVectorizer()
text = tfidf_vect.transform([(original_text)])

# Kita prediksi sentimennya
result = model.predict(text)[0]
print("Sentiment:")
print()
print(result)

Sentiment:

negative


In [13]:
original_text =  '''
Ibu saya
'''

# Feature Extraction
count_vect = TfidfVectorizer()
text = tfidf_vect.transform([(original_text)])

# Kita prediksi sentimennya
result = model.predict(text)[0]
print("Sentiment:")
print()
print(result)

Sentiment:

negative


In [14]:
model.predict_proba(text)

array([[0.71640952, 0.13489728, 0.14869321]])

In [15]:
# Untuk lebih menyakinkan lagi, kita juga bisa melakukan "Cross Validation"

kf = KFold(n_splits=5,random_state=42,shuffle=True)

accuracies = []

y = classes

for iteration, data in enumerate(kf.split(x), start=1):

    data_train   = x[data[0]]
    target_train = y[data[0]]

    data_test    = x[data[1]]
    target_test  = y[data[1]]

    clf = MLPClassifier()
    # clf = svm.SVC(gamma=0.01, C=100., probability=True)
    clf.fit(data_train,target_train)

    preds = clf.predict(data_test)

    # for the current fold only    
    accuracy = accuracy_score(target_test,preds)

    print("Training ke-", iteration)
    print(classification_report(target_test,preds))
    print("======================================================")

    accuracies.append(accuracy)

# this is the average accuracy over all folds
average_accuracy = np.mean(accuracies)

print()
print()
print()
print("Rata-rata Accuracy: ", average_accuracy)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.78      0.80      0.79       680
     neutral       0.83      0.71      0.77       239
    positive       0.88      0.90      0.89      1281

    accuracy                           0.85      2200
   macro avg       0.83      0.80      0.81      2200
weighted avg       0.85      0.85      0.84      2200

Training ke- 2
              precision    recall  f1-score   support

    negative       0.80      0.75      0.77       706
     neutral       0.74      0.71      0.73       220
    positive       0.87      0.91      0.89      1274

    accuracy                           0.84      2200
   macro avg       0.80      0.79      0.80      2200
weighted avg       0.83      0.84      0.83      2200

Training ke- 3
              precision    recall  f1-score   support

    negative       0.79      0.79      0.79       682
     neutral       0.84      0.75      0.79       215
    positive       0.89      0